Importing the Dataset using kagglehub

In [ ]:
import kagglehub

path = kagglehub.dataset_download("buraktaci/cerebrovascular-lesions")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'cerebrovascular-lesions' dataset.
Path to dataset files: /kaggle/input/cerebrovascular-lesions


Preprocessing the Image

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

nested_folder_name = 'cerebrovascular lesions'

if nested_folder_name in os.listdir(path):
    path = os.path.join(path, nested_folder_name)
    print(f"Path corrected to nested folder: {path}")
else:
    print(f"Could not find expected subfolder '{nested_folder_name}'.")

output_root = '/content/preprocessed_cerebrovascular_lesions'
os.makedirs(output_root, exist_ok=True)

target_size = (128, 128)

print("\nStarting Preprocessing...")

for class_name in sorted(os.listdir(path)):
    class_in = os.path.join(path, class_name)
    class_out = os.path.join(output_root, class_name)

    if not os.path.isdir(class_in):
        continue

    os.makedirs(class_out, exist_ok=True)

    print(f'Processing: {class_name}')
    for filename in tqdm(os.listdir(class_in)):
        if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
            img_path = os.path.join(class_in, filename)

            img = cv2.imread(img_path)
            if img is None:
                continue

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, target_size)
            normalized_float = (resized / 255.0)

            save_img = (normalized_float * 255).astype(np.uint8)

            save_path = os.path.join(class_out, filename)
            cv2.imwrite(save_path, save_img)

Path corrected to nested folder: /kaggle/input/cerebrovascular-lesions/cerebrovascular lesions

Starting Preprocessing...
Processing: Acute ischemic infarction


100%|██████████| 1119/1119 [00:26<00:00, 42.71it/s]


Processing: Chronic ischemic infarction


100%|██████████| 1008/1008 [00:23<00:00, 43.54it/s]


Processing: Epidural hemorrhage


...

100%|██████████| 1080/1080 [00:25<00:00, 41.64it/s]


Processing: Subdural hemorrhage


100%|██████████| 1177/1177 [00:31<00:00, 37.93it/s]


Processing: Ventricular hemorrhage


100%|██████████| 1020/1020 [00:25<00:00, 39.46it/s]


Verifying if the images are preprocessed

In [ ]:
import os
import cv2
import numpy as np

preprocessed_root = '/content/preprocessed_cerebrovascular_lesions'

print("Verifying preprocessed images...")

all_grayscale = True
all_correct_size = True
all_normalized = True
target_size = (128, 128)

for class_name in sorted(os.listdir(preprocessed_root)):
    class_path = os.path.join(preprocessed_root, class_name)

    if not os.path.isdir(class_path):
        continue

    print(f"Checking class: {class_name}")
    for filename in os.listdir(class_path):
        if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
            img_path = os.path.join(class_path, filename)

            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

            if img is None:
                print(f"Could not read image: {img_path}")
                continue

            if len(img.shape) == 3 and img.shape[2] == 3:
                 all_grayscale = False
                 print(f"Image is not grayscale: {img_path}")

            if img.shape[:2][::-1] != target_size:
                all_correct_size = False
                print(f"Image has incorrect dimensions: {img_path} - {img.shape[:2][::-1]}")

            if img.dtype != np.uint8 or np.min(img) < 0 or np.max(img) > 255:
                all_normalized = False
                print(f"Image is not normalized (0-255 uint8): {img_path}")

print("\nVerification Results:")
print(f"All images are grayscale: {all_grayscale}")
print(f"All images have correct dimensions ({target_size[0]}x{target_size[1]}): {all_correct_size}")
print(f"All images are normalized (0-255 uint8): {all_normalized}")

Verifying preprocessed images...
Checking class: Acute ischemic infarction
Checking class: Chronic ischemic infarction
Checking class: Epidural hemorrhage
Checking class: Healthy Control
Checking class: Mixed cerebrovascular disease
Checking class: Parenchymal hemorrhage
Checking class: Subarachnoid arachnoid hemorrhage
Checking class: Subdural effusion
Checking class: Subdural hemorrhage
Checking class: Ventricular hemorrhage

Verification Results:
All images are grayscale: True
All images have correct dimensions (128x128): True
All images are normalized (0-255 uint8): True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import hog, local_binary_pattern, graycomatrix, graycoprops
from skimage.filters import gabor

Radiomics Feature Extraction

In [ ]:
def extract_radiomics_features(img):
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (128, 128))
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')

    # 1. HOG
    hog_features, _ = hog(img, orientations=9, pixels_per_cell=(8, 8),
                          cells_per_block=(2, 2), block_norm='L2-Hys',
                          visualize=True, feature_vector=True)

    # 2. LBP
    lbp = local_binary_pattern(img, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 59), range=(0, 58))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-6)

    # 3. GLCM
    glcm = graycomatrix(img, distances=[1, 2, 3],
                        angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                        levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast').ravel()
    correlation = graycoprops(glcm, 'correlation').ravel()
    energy = graycoprops(glcm, 'energy').ravel()
    homogeneity = graycoprops(glcm, 'homogeneity').ravel()
    glcm_features = np.hstack([contrast, correlation, energy, homogeneity])

    # 4. Haralick-like descriptors (summary stats from GLCM)
    haralick_features = [
        glcm.mean(), glcm.var(), contrast.mean(),
        correlation.mean(), energy.mean(), homogeneity.mean()
    ]

    # 5. Gabor Filters
    gabor_features = []
    frequencies = (0.1, 0.3, 0.5)
    for theta in (0, np.pi/4, np.pi/2, 3*np.pi/4):
        for freq in frequencies:
            filt_real, filt_imag = gabor(img, frequency=freq, theta=theta)
            gabor_features.append(filt_real.mean())
            gabor_features.append(filt_real.var())
    gabor_features = np.array(gabor_features)

    # Combine all
    return np.hstack([hog_features, lbp_hist, glcm_features,
                      haralick_features, gabor_features])

Saving the Extraced features into CSV files

In [ ]:
def process_dataset(dataset_path, output_csv='radiomics_features.csv', save_every=2500):
  data, labels = [], []
  count = 0
  file_index = 1

  for root, dirs, files in os.walk(dataset_path):
    for file in files:
      if file.lower().endswith(('.png', '.jpg', '.jpeg')):
        path = os.path.join(root, file)
        label = os.path.basename(root)

        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        if img is None:
          print(f"Could not read image at {path}. Skipping file.")
          continue

        features = extract_radiomics_features(img)

        data.append(features)
        labels.append(label)
        count += 1

        if count % 500 == 0:
          print(f"Processed {count} images...")

          # Save partial CSV every N images
        if count % save_every == 0:
          df = pd.DataFrame(data)
          df['label'] = labels
          part_name = f"{output_csv.split('.')[0]}_pt{file_index}.csv"
          df.to_csv(part_name, index=False)
          print(f"Saved {part_name} after {count} images.")
          data, labels = [], []
          file_index += 1

    # Save remaining
    if data:
        df = pd.DataFrame(data)
        df['label'] = labels
        part_name = f"{output_csv.split('.')[0]}_pt{file_index}.csv"
        df.to_csv(part_name, index=False)
        print(f"Saved final part: {part_name}")


In [ ]:
dataset_path = "/content/drive/MyDrive/paneer/preprocessed_cerebrovascular_lesions"
process_dataset(dataset_path, output_csv="radiomics_features.csv")

Combining the CSV files and shuffling it into 1 CSV file

In [ ]:
import pandas as pd
import glob

path = "/content/drive/MyDrive/paneer/csv_files/part_csvs"

csv_files = glob.glob(path + "/*.csv")

combined_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
combined_shuff_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

combined_shuff_df.to_csv('/content/drive/MyDrive/paneer/csv_files/radiomics_features_full_shuffled.csv')

Verifying the count

In [ ]:
combined_shuff_df['label'].value_counts()

,count
label,
Mixed cerebrovascular disease,1191
Subdural hemorrhage,1177
Subarachnoid arachnoid hemorrhage,1166
Acute ischemic infarction,1119
Subdural effusion,1080
Parenchymal hemorrhage,1077
Ventricular hemorrhage,1020
Healthy Control,1016
Epidural hemorrhage,1011


In [ ]:
import pandas as pd

shuffled_df = pd.read_csv('/content/drive/MyDrive/paneer/csv_files/radiomics_features_full_shuffled.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
X = shuffled_df.drop(columns=['label']).values
y = shuffled_df['label']

Saving the Label Encoder as a .pkl file

In [ ]:
from sklearn.preprocessing import LabelEncoder
import joblib

le = LabelEncoder()
y_encoded = le.fit_transform(y)

joblib.dump(le, '/content/drive/MyDrive/paneer/label_encoder.pkl')

['/content/drive/MyDrive/paneer/label_encoder.pkl']

In [ ]:
for i, class_name in enumerate(le.classes_):
    print(f"Class {class_name} -> Label {i}")

Class Acute ischemic infarction -> Label 0
Class Chronic ischemic infarction -> Label 1
Class Epidural hemorrhage -> Label 2
Class Healthy Control -> Label 3
Class Mixed cerebrovascular disease -> Label 4
Class Parenchymal hemorrhage -> Label 5
Class Subarachnoid arachnoid hemorrhage -> Label 6
Class Subdural effusion -> Label 7
Class Subdural hemorrhage -> Label 8
Class Ventricular hemorrhage -> Label 9


Load the Label Encoder later

In [ ]:
from sklearn.preprocessing import LabelEncoder
import joblib

le = joblib.load('/content/drive/MyDrive/paneer/label_encoder.pkl')

y_encoded = le.transform(y)
#y_pred_original = le.inverse_transform(y_pred)

In [ ]:
!pip install optuna --quiet

automated search for transforming 8000-feature dataset into an optimized feature set.

In [ ]:
import numpy as np
import optuna
import warnings
import time
import gc
import pandas as pd
import os
import csv

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

SHARED_PATH = "/content/drive/MyDrive/paneer"
os.makedirs(f"{SHARED_PATH}/csv_files", exist_ok=True)

CSV_LOG = f"{SHARED_PATH}/csv_files/optuna_trials_log.csv"
if not os.path.exists(CSV_LOG):
    with open(CSV_LOG, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["trial_number", "selector_threshold", "learning_rate", "max_depth",
                         "fold1", "fold2", "fold3", "mean_cv"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)
gc.collect()
print(f"Data split: X_train={X_train.shape}, X_test={X_test.shape}")

N_COMPONENTS_IPCA = 256

def objective_pipeline(trial):
    selector_threshold = trial.suggest_categorical("selector_threshold", ["median", "mean"])
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2)
    max_depth = trial.suggest_int("max_depth", 4, 8)

    xgb_params = {
        "n_estimators": 100,
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "objective": "multi:softmax",
        "num_class": len(np.unique(y_train)),
        "random_state": 42,
        "n_jobs": 1,
        "use_label_encoder": False,
        "eval_metric": "mlogloss"
    }

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = []

    print(f"\n=== Trial {trial.number} started: selector_threshold={selector_threshold}, "
          f"learning_rate={learning_rate:.3f}, max_depth={max_depth} ===")

    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), start=1):
        print(f"  --> Fold {fold}/3 processing...")
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        # Scale
        scaler = StandardScaler().fit(X_tr)
        X_tr_scaled = scaler.transform(X_tr)
        X_val_scaled = scaler.transform(X_val)

        # Feature selection
        selector = SelectFromModel(
            ExtraTreesClassifier(n_estimators=50, random_state=42, n_jobs=1),
            threshold=selector_threshold
        ).fit(X_tr_scaled, y_tr)
        X_tr_sel = selector.transform(X_tr_scaled)
        X_val_sel = selector.transform(X_val_scaled)

        if X_tr_sel.shape[1] == 0:
            print("    - WARNING: No features selected in this fold. Pruning trial.")
            return 0.0

        # Incremental PCA
        ipca = IncrementalPCA(n_components=min(N_COMPONENTS_IPCA, X_tr_sel.shape[1]), batch_size=256).fit(X_tr_sel)
        X_tr_pca = ipca.transform(X_tr_sel)
        X_val_pca = ipca.transform(X_val_sel)

        # Train XGBoost
        model = xgb.XGBClassifier(**xgb_params).fit(X_tr_pca, y_tr)
        fold_acc = accuracy_score(y_val, model.predict(X_val_pca))
        cv_scores.append(fold_acc)
        print(f"    - Fold {fold} accuracy: {fold_acc:.4f}")

        # Free memory
        del X_tr, X_val, X_tr_scaled, X_val_scaled, X_tr_sel, X_val_sel, X_tr_pca, X_val_pca
        gc.collect()

    mean_cv = np.mean(cv_scores)
    print(f"=== Trial {trial.number} finished. Mean CV accuracy: {mean_cv:.4f} ===")

    with open(CSV_LOG, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([trial.number, selector_threshold, learning_rate, max_depth,
                         cv_scores[0], cv_scores[1], cv_scores[2], mean_cv])

    return mean_cv

study = optuna.create_study(direction="maximize")
study.optimize(objective_pipeline, n_trials=30)

print("\nOptuna finished!")
print("Best CV accuracy:", study.best_value)
print("Best params:", study.best_params)
best_params = study.best_params

print("\n--- Building final pipeline on full dataset ---")
final_scaler = StandardScaler().fit(X_train)
X_train_scaled = final_scaler.transform(X_train)
X_test_scaled = final_scaler.transform(X_test)

final_selector = SelectFromModel(
    ExtraTreesClassifier(n_estimators=50, random_state=42),
    threshold=best_params["selector_threshold"]
).fit(X_train_scaled, y_train)
X_train_sel = final_selector.transform(X_train_scaled)
X_test_sel = final_selector.transform(X_test_scaled)

final_ipca = IncrementalPCA(n_components=min(N_COMPONENTS_IPCA, X_train_sel.shape[1]), batch_size=256).fit(X_train_sel)
X_train_final = final_ipca.transform(X_train_sel)
X_test_final = final_ipca.transform(X_test_sel)

pd.DataFrame(X_train_final).to_csv(f"{SHARED_PATH}/csv_files/X_train_final.csv", index=False)
pd.DataFrame(X_test_final).to_csv(f"{SHARED_PATH}/csv_files/X_test_final.csv", index=False)
pd.DataFrame(y_train, columns=['label']).to_csv(f"{SHARED_PATH}/csv_files/y_train.csv", index=False)
pd.DataFrame(y_test, columns=['label']).to_csv(f"{SHARED_PATH}/csv_files/y_test.csv", index=False)

print("\n✅ Final datasets saved!")
print(f"X_train_final shape: {X_train_final.shape}, X_test_final shape: {X_test_final.shape}")
print(f"Trial log saved at: {CSV_LOG}")


[I 2025-10-14 13:00:20,383] A new study created in memory with name: no-name-790e6034-cf60-4e9b-9149-5ccaef2c8e31


Data split: X_train=(8692, 8237), X_test=(2173, 8237)

=== Trial 0 started: selector_threshold=mean, learning_rate=0.095, max_depth=8 ===
  --> Fold 1/3 processing...
    - Fold 1 accuracy: 0.8761
  --> Fold 2/3 processing...
    - Fold 2 accuracy: 0.8985
  --> Fold 3/3 processing...


[I 2025-10-14 13:09:15,542] Trial 0 finished with value: 0.8888647886936972 and parameters: {'selector_threshold': 'mean', 'learning_rate': 0.09460147454354628, 'max_depth': 8}. Best is trial 0 with value: 0.8888647886936972.


    - Fold 3 accuracy: 0.8920
=== Trial 0 finished. Mean CV accuracy: 0.8889 ===

=== Trial 1 started: selector_threshold=mean, learning_rate=0.058, max_depth=7 ===
  --> Fold 1/3 processing...
    - Fold 1 accuracy: 0.8489
  --> Fold 2/3 processing...
    - Fold 2 accuracy: 0.8740
  --> Fold 3/3 processing...


[I 2025-10-14 13:17:38,379] Trial 1 finished with value: 0.8643598134525781 and parameters: {'selector_threshold': 'mean', 'learning_rate': 0.057712122939594226, 'max_depth': 7}. Best is trial 0 with value: 0.8888647886936972.


...

    - Fold 3 accuracy: 0.8509
=== Trial 27 finished. Mean CV accuracy: 0.8496 ===

=== Trial 28 started: selector_threshold=median, learning_rate=0.200, max_depth=6 ===
  --> Fold 1/3 processing...
    - Fold 1 accuracy: 0.8879
  --> Fold 2/3 processing...
    - Fold 2 accuracy: 0.9082
  --> Fold 3/3 processing...


[I 2025-10-14 15:54:36,168] Trial 28 finished with value: 0.8985286096315498 and parameters: {'selector_threshold': 'median', 'learning_rate': 0.19958865737541653, 'max_depth': 6}. Best is trial 11 with value: 0.9034753831395035.


    - Fold 3 accuracy: 0.8996
=== Trial 28 finished. Mean CV accuracy: 0.8985 ===

=== Trial 29 started: selector_threshold=mean, learning_rate=0.110, max_depth=8 ===
  --> Fold 1/3 processing...
    - Fold 1 accuracy: 0.8834
  --> Fold 2/3 processing...
    - Fold 2 accuracy: 0.8975
  --> Fold 3/3 processing...


[I 2025-10-14 16:02:46,240] Trial 29 finished with value: 0.8923158016522966 and parameters: {'selector_threshold': 'mean', 'learning_rate': 0.11048379809432742, 'max_depth': 8}. Best is trial 11 with value: 0.9034753831395035.


    - Fold 3 accuracy: 0.8961
=== Trial 29 finished. Mean CV accuracy: 0.8923 ===

Optuna finished!
Best CV accuracy: 0.9034753831395035
Best params: {'selector_threshold': 'median', 'learning_rate': 0.19843366032826454, 'max_depth': 7}

--- Building final pipeline on full dataset ---

✅ Final datasets saved!
X_train_final shape: (8692, 256), X_test_final shape: (2173, 256)
Trial log saved at: /content/drive/MyDrive/paneer/csv_files/optuna_trials_log.csv


**Model Training** (w/out Naive Bayes and Bayesian Network)

In [ ]:
import pandas as pd
import numpy as np
import optuna
import warnings
import time
import joblib
import os

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb

from sklearn.model_selection import cross_val_score

warnings.filterwarnings('ignore')

def tune_and_save_model(model_name: str):

    print(f"--- 1. Loading Optimized Data for [{model_name}] ---")
    try:
        X_train_final = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/X_train_final.csv").values
        y_train = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/y_train.csv")['label'].values
        print("Data loaded successfully.\n")
    except FileNotFoundError:
        print("Error: Could not find data files. Please run Script 1 first.")
        return

    print(f"--- 2. Preparing Tuning Objective for [{model_name}] ---")
    models_to_tune = {
        "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42),
        "KNN": KNeighborsClassifier(),
        "SVM": SVC(random_state=42),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42, n_jobs=-1),
        "AdaBoost": AdaBoostClassifier(random_state=42),
        "MLP": MLPClassifier(max_iter=500, random_state=42, early_stopping=True),
        "XGBoost": xgb.XGBClassifier(random_state=42, n_jobs=-1, use_label_encoder=False, eval_metric='mlogloss')
    }

    def objective_model_tuning(trial):
        if model_name == "Logistic Regression": params = {'C': trial.suggest_float('C', 1e-3, 1e3, log=True)}
        elif model_name == "KNN": params = {'n_neighbors': trial.suggest_int('n_neighbors', 3, 31)}
        elif model_name == "SVM": params = {'C': trial.suggest_float('C', 1e-2, 1e2, log=True), 'gamma': trial.suggest_float('gamma', 1e-3, 1e-1, log=True)}
        elif model_name == "Decision Tree": params = {'max_depth': trial.suggest_int('max_depth', 3, 50), 'min_samples_split': trial.suggest_int('min_samples_split', 2, 32)}
        elif model_name == "Random Forest": params = {'n_estimators': trial.suggest_int('n_estimators', 100, 800), 'max_depth': trial.suggest_int('max_depth', 10, 50)}
        elif model_name == "AdaBoost": params = {'n_estimators': trial.suggest_int('n_estimators', 50, 500), 'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0)}
        elif model_name == "MLP": params = {'alpha': trial.suggest_float('alpha', 1e-5, 1e-1, log=True), 'learning_rate_init': trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True)}
        elif model_name == "XGBoost": params = {"n_estimators": 500, "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3), "max_depth": trial.suggest_int("max_depth", 3, 10)}

        model = models_to_tune[model_name].set_params(**params)
        return cross_val_score(model, X_train_final, y_train, n_jobs=-1, cv=5).mean()

    def print_callback(study, trial):
        print(f"  > Trial {trial.number} finished. Score: {trial.value:.4f}. Best score: {study.best_value:.4f}")

    print(f"--- 3. Starting Optuna Study for [{model_name}] ---")
    start_time = time.time()
    study = optuna.create_study(direction="maximize")

    study.optimize(
        objective_model_tuning,
        n_trials=20,
        callbacks=[print_callback]
    )

    print("\n--- 4. Training and Saving the Best Model ---")
    best_params = study.best_params
    print(f"Best parameters found: {best_params}")

    best_model = models_to_tune[model_name].set_params(**best_params)
    best_model.fit(X_train_final, y_train)

    os.makedirs("/content/drive/MyDrive/paneer/models", exist_ok=True)
    save_path = f"/content/drive/MyDrive/paneer/models/{model_name.replace(' ', '_')}_best_model.pkl"
    joblib.dump(best_model, save_path)
    print(f"[{model_name}] saved successfully at: {save_path}\n")


In [ ]:
tune_and_save_model("Logistic Regression")

--- 1. Loading Optimized Data for [Logistic Regression] ---


[I 2025-10-15 10:43:24,515] A new study created in memory with name: no-name-747ecb36-9a69-4f17-80b1-8d96dc58662e


Data loaded successfully.

--- 2. Preparing Tuning Objective for [Logistic Regression] ---
--- 3. Starting Optuna Study for [Logistic Regression] ---


[I 2025-10-15 10:45:08,905] Trial 0 finished with value: 0.7356170067185419 and parameters: {'C': 43.90162235960686}. Best is trial 0 with value: 0.7356170067185419.


  > Trial 0 finished. Score: 0.7356. Best score: 0.7356


...

  > Trial 17 finished. Score: 0.7731. Best score: 0.7737


[I 2025-10-15 10:57:18,202] Trial 18 finished with value: 0.7732384589373547 and parameters: {'C': 0.0030405677124782453}. Best is trial 1 with value: 0.7736985596129146.


  > Trial 18 finished. Score: 0.7732. Best score: 0.7737


[I 2025-10-15 10:57:38,939] Trial 19 finished with value: 0.7623089999874271 and parameters: {'C': 0.06340705397721486}. Best is trial 1 with value: 0.7736985596129146.


  > Trial 19 finished. Score: 0.7623. Best score: 0.7737

--- 4. Training and Saving the Best Model ---
Best parameters found: {'C': 0.004626444834674163}
✅ [Logistic Regression] saved successfully at: /content/drive/MyDrive/paneer/models/Logistic_Regression_best_model.pkl



In [ ]:
tune_and_save_model("KNN")

--- 1. Loading Optimized Data for [KNN] ---


[I 2025-10-15 11:02:18,550] A new study created in memory with name: no-name-932d6215-85ef-4a81-9804-ed2bab596290


Data loaded successfully.

--- 2. Preparing Tuning Objective for [KNN] ---
--- 3. Starting Optuna Study for [KNN] ---


[I 2025-10-15 11:02:20,106] Trial 0 finished with value: 0.9389091120844419 and parameters: {'n_neighbors': 4}. Best is trial 0 with value: 0.9389091120844419.


  > Trial 0 finished. Score: 0.9389. Best score: 0.9389


...

  > Trial 17 finished. Score: 0.9501. Best score: 0.9501


[I 2025-10-15 11:02:50,139] Trial 18 finished with value: 0.9020922570343524 and parameters: {'n_neighbors': 8}. Best is trial 12 with value: 0.9500682574208025.


  > Trial 18 finished. Score: 0.9021. Best score: 0.9501


[I 2025-10-15 11:02:51,646] Trial 19 finished with value: 0.8066008201478171 and parameters: {'n_neighbors': 21}. Best is trial 12 with value: 0.9500682574208025.


  > Trial 19 finished. Score: 0.8066. Best score: 0.9501

--- 4. Training and Saving the Best Model ---
Best parameters found: {'n_neighbors': 3}
✅ [KNN] saved successfully at: /content/drive/MyDrive/paneer/models/KNN_best_model.pkl



In [ ]:
tune_and_save_model("SVM")

--- 1. Loading Optimized Data for [SVM] ---


[I 2025-10-15 11:05:43,550] A new study created in memory with name: no-name-97991f90-1843-4712-85b2-333431a35ad5


Data loaded successfully.

--- 2. Preparing Tuning Objective for [SVM] ---
--- 3. Starting Optuna Study for [SVM] ---


[I 2025-10-15 11:07:11,743] Trial 0 finished with value: 0.9389085827006646 and parameters: {'C': 16.916515572790665, 'gamma': 0.002294206871076437}. Best is trial 0 with value: 0.9389085827006646.


  > Trial 0 finished. Score: 0.9389. Best score: 0.9389


...

  > Trial 17 finished. Score: 0.9619. Best score: 0.9635


[I 2025-10-15 11:33:42,790] Trial 18 finished with value: 0.953979675633325 and parameters: {'C': 9.33167462986442, 'gamma': 0.0017233436204535791}. Best is trial 10 with value: 0.963529229594406.


  > Trial 18 finished. Score: 0.9540. Best score: 0.9635


[I 2025-10-15 11:35:12,148] Trial 19 finished with value: 0.7986644970754855 and parameters: {'C': 0.7414858795964644, 'gamma': 0.004446200508174477}. Best is trial 10 with value: 0.963529229594406.


  > Trial 19 finished. Score: 0.7987. Best score: 0.9635

--- 4. Training and Saving the Best Model ---
Best parameters found: {'C': 4.775510758722773, 'gamma': 0.0010226747698128193}
✅ [SVM] saved successfully at: /content/drive/MyDrive/paneer/models/SVM_best_model.pkl



In [ ]:
tune_and_save_model("Decision Tree")

--- 1. Loading Optimized Data for [Decision Tree] ---


[I 2025-10-15 14:16:50,606] A new study created in memory with name: no-name-63d00692-21aa-416d-a2f0-8f7339901cb5


Data loaded successfully.

--- 2. Preparing Tuning Objective for [Decision Tree] ---
--- 3. Starting Optuna Study for [Decision Tree] ---


[I 2025-10-15 14:17:11,695] Trial 0 finished with value: 0.5011538581158834 and parameters: {'max_depth': 13, 'min_samples_split': 7}. Best is trial 0 with value: 0.5011538581158834.


  > Trial 0 finished. Score: 0.5012. Best score: 0.5012


...

  > Trial 17 finished. Score: 0.5361. Best score: 0.6030


[I 2025-10-15 14:22:56,025] Trial 18 finished with value: 0.5992877803004385 and parameters: {'max_depth': 43, 'min_samples_split': 5}. Best is trial 11 with value: 0.602967328418446.


  > Trial 18 finished. Score: 0.5993. Best score: 0.6030


[I 2025-10-15 14:23:15,348] Trial 19 finished with value: 0.5616683794437632 and parameters: {'max_depth': 20, 'min_samples_split': 14}. Best is trial 11 with value: 0.602967328418446.


  > Trial 19 finished. Score: 0.5617. Best score: 0.6030

--- 4. Training and Saving the Best Model ---
Best parameters found: {'max_depth': 34, 'min_samples_split': 2}
✅ [Decision Tree] saved successfully at: /content/drive/MyDrive/paneer/models/Decision_Tree_best_model.pkl



In [ ]:
tune_and_save_model("Random Forest")

--- 1. Loading Optimized Data for [Random Forest] ---


[I 2025-10-15 12:55:21,679] A new study created in memory with name: no-name-241b2884-e7f4-483f-b74d-03f038acd183


Data loaded successfully.

--- 2. Preparing Tuning Objective for [Random Forest] ---
--- 3. Starting Optuna Study for [Random Forest] ---


[I 2025-10-15 13:00:11,719] Trial 0 finished with value: 0.9164738937698808 and parameters: {'n_estimators': 433, 'max_depth': 12}. Best is trial 0 with value: 0.9164738937698808.


  > Trial 0 finished. Score: 0.9165. Best score: 0.9165


...

  > Trial 17 finished. Score: 0.9394. Best score: 0.9403


[I 2025-10-15 15:04:13,200] Trial 18 finished with value: 0.93534179332725 and parameters: {'n_estimators': 311, 'max_depth': 47}. Best is trial 7 with value: 0.9402890170732885.


  > Trial 18 finished. Score: 0.9353. Best score: 0.9403


[I 2025-10-15 15:11:12,013] Trial 19 finished with value: 0.9402890832462608 and parameters: {'n_estimators': 525, 'max_depth': 36}. Best is trial 19 with value: 0.9402890832462608.


  > Trial 19 finished. Score: 0.9403. Best score: 0.9403

--- 4. Training and Saving the Best Model ---
Best parameters found: {'n_estimators': 525, 'max_depth': 36}
✅ [Random Forest] saved successfully at: /content/drive/MyDrive/paneer/models/Random_Forest_best_model.pkl



In [ ]:
tune_and_save_model("AdaBoost")

--- 1. Loading Optimized Data for [AdaBoost] ---


[I 2025-10-15 14:48:38,745] A new study created in memory with name: no-name-14d21633-8155-4b82-ace8-f54863c7e0b8


Data loaded successfully.

--- 2. Preparing Tuning Objective for [AdaBoost] ---
--- 3. Starting Optuna Study for [AdaBoost] ---


[I 2025-10-15 14:55:09,944] Trial 0 finished with value: 0.3115491026614108 and parameters: {'n_estimators': 216, 'learning_rate': 0.30240424717722986}. Best is trial 0 with value: 0.3115491026614108.


  > Trial 0 finished. Score: 0.3115. Best score: 0.3115


...

  > Trial 17 finished. Score: 0.4382. Best score: 0.4429


[I 2025-10-15 17:54:01,746] Trial 18 finished with value: 0.4348810309219682 and parameters: {'n_estimators': 453, 'learning_rate': 0.8525927101961053}. Best is trial 15 with value: 0.44293395077127906.


  > Trial 18 finished. Score: 0.4349. Best score: 0.4429


[I 2025-10-15 18:07:28,787] Trial 19 finished with value: 0.42130471925785684 and parameters: {'n_estimators': 450, 'learning_rate': 0.6527667206649479}. Best is trial 15 with value: 0.44293395077127906.


  > Trial 19 finished. Score: 0.4213. Best score: 0.4429

--- 4. Training and Saving the Best Model ---
Best parameters found: {'n_estimators': 480, 'learning_rate': 0.7503341904031118}
✅ [AdaBoost] saved successfully at: /content/drive/MyDrive/paneer/models/AdaBoost_best_model.pkl



In [ ]:
tune_and_save_model("MLP")

--- 1. Loading Optimized Data for [MLP] ---


[I 2025-10-15 14:31:33,338] A new study created in memory with name: no-name-5fc384fa-21cf-46f9-888a-3cea9457706a


Data loaded successfully.

--- 2. Preparing Tuning Objective for [MLP] ---
--- 3. Starting Optuna Study for [MLP] ---


[I 2025-10-15 14:32:10,960] Trial 0 finished with value: 0.9129066411856608 and parameters: {'alpha': 0.0010223115894471173, 'learning_rate_init': 0.00025054117563278}. Best is trial 0 with value: 0.9129066411856608.


  > Trial 0 finished. Score: 0.9129. Best score: 0.9129


...

  > Trial 17 finished. Score: 0.9265. Best score: 0.9336


[I 2025-10-15 14:37:10,798] Trial 18 finished with value: 0.9224570553953801 and parameters: {'alpha': 0.038316797283696274, 'learning_rate_init': 0.0014014690532320974}. Best is trial 16 with value: 0.9336154728290469.


  > Trial 18 finished. Score: 0.9225. Best score: 0.9336


[I 2025-10-15 14:37:21,106] Trial 19 finished with value: 0.9226861462250635 and parameters: {'alpha': 0.003571104038414152, 'learning_rate_init': 0.004646715701858819}. Best is trial 16 with value: 0.9336154728290469.


  > Trial 19 finished. Score: 0.9227. Best score: 0.9336

--- 4. Training and Saving the Best Model ---
Best parameters found: {'alpha': 0.08447161591584045, 'learning_rate_init': 0.005133416724000536}
✅ [MLP] saved successfully at: /content/drive/MyDrive/paneer/models/MLP_best_model.pkl



In [ ]:
tune_and_save_model("XGBoost")

--- 1. Loading Optimized Data for [XGBoost] ---


[I 2025-10-16 05:18:50,539] A new study created in memory with name: no-name-e87b40d4-7845-4e7b-883e-f701c49520da


Data loaded successfully.

--- 2. Preparing Tuning Objective for [XGBoost] ---
--- 3. Starting Optuna Study for [XGBoost] ---


[I 2025-10-16 05:28:49,962] Trial 0 finished with value: 0.9268286404564348 and parameters: {'learning_rate': 0.1643228615169814, 'max_depth': 4}. Best is trial 0 with value: 0.9268286404564348.


  > Trial 0 finished. Score: 0.9268. Best score: 0.9268


...

  > Trial 17 finished. Score: 0.9269. Best score: 0.9272


[I 2025-10-16 09:51:25,038] Trial 18 finished with value: 0.9259082405863985 and parameters: {'learning_rate': 0.19534989534517103, 'max_depth': 6}. Best is trial 15 with value: 0.9271736001604033.


  > Trial 18 finished. Score: 0.9259. Best score: 0.9272


[I 2025-10-16 10:16:57,257] Trial 19 finished with value: 0.9211906370538205 and parameters: {'learning_rate': 0.07726752440143321, 'max_depth': 10}. Best is trial 15 with value: 0.9271736001604033.


  > Trial 19 finished. Score: 0.9212. Best score: 0.9272

--- 4. Training and Saving the Best Model ---
Best parameters found: {'learning_rate': 0.09576639896717229, 'max_depth': 6}
✅ [XGBoost] saved successfully at: /content/drive/MyDrive/paneer/models/XGBoost_best_model.pkl



Naive Bayes Training

In [ ]:
import pandas as pd
import numpy as np
import optuna
import warnings
import joblib
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

warnings.filterwarnings('ignore')

X_train = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/X_train_final.csv").values
y_train = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/y_train.csv")['label'].values

def objective(trial):
    var_smoothing = trial.suggest_float('var_smoothing', 1e-12, 1e-6, log=True)
    model = GaussianNB(var_smoothing=var_smoothing)
    return cross_val_score(model, X_train, y_train, cv=5, n_jobs=-1).mean()

def print_callback(study, trial):
    print(f"Trial {trial.number} finished. Score: {trial.value:.4f}. Best score: {study.best_value:.4f}")

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, callbacks=[print_callback])

print("\nBest var_smoothing:", study.best_params)

best_model = GaussianNB(var_smoothing=study.best_params['var_smoothing'])
best_model.fit(X_train, y_train)

os.makedirs("/content/drive/MyDrive/paneer/models", exist_ok=True)
save_path = "/content/drive/MyDrive/paneer/models/Naive_Bayes_best_model.pkl"
joblib.dump(best_model, save_path)
print(f"Naive Bayes model saved at: {save_path}")

[I 2025-10-16 10:24:30,835] A new study created in memory with name: no-name-f6adf4ba-0015-4653-9a2b-1e6a951a53b2
[I 2025-10-16 10:24:33,885] Trial 0 finished with value: 0.5761558267618058 and parameters: {'var_smoothing': 2.9240467333093715e-09}. Best is trial 0 with value: 0.5761558267618058.


Trial 0 finished. Score: 0.5762. Best score: 0.5762


[I 2025-10-16 10:24:34,271] Trial 1 finished with value: 0.5761558267618058 and parameters: {'var_smoothing': 1.306049753319959e-11}. Best is trial 0 with value: 0.5761558267618058.


Trial 1 finished. Score: 0.5762. Best score: 0.5762


[I 2025-10-16 10:24:34,657] Trial 2 finished with value: 0.5761558267618058 and parameters: {'var_smoothing': 9.52264243503519e-12}. Best is trial 0 with value: 0.5761558267618058.


...

Trial 17 finished. Score: 0.5762. Best score: 0.5762


[I 2025-10-16 10:24:40,789] Trial 18 finished with value: 0.5761558267618058 and parameters: {'var_smoothing': 1.0508194251547628e-11}. Best is trial 0 with value: 0.5761558267618058.


Trial 18 finished. Score: 0.5762. Best score: 0.5762


[I 2025-10-16 10:24:41,338] Trial 19 finished with value: 0.5761558267618058 and parameters: {'var_smoothing': 4.153348405098605e-10}. Best is trial 0 with value: 0.5761558267618058.


Trial 19 finished. Score: 0.5762. Best score: 0.5762

Best var_smoothing: {'var_smoothing': 2.9240467333093715e-09}
Naive Bayes model saved at: /content/drive/MyDrive/paneer/models/Naive_Bayes_best_model.pkl


Bayesian Network Training

In [ ]:
!pip install bnlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.4/190.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 7.5 MB/s eta 0:00:00
  Created wheel for semopy: filename=semopy-2.3.11-py3-none-any.whl size=1659682 sha256=ee97943fdda2d5b2349b58d18d2126ebaa3a8c5d4ba2fd4cdab0f49ff1a26812
  Stored in directory: /root/.cache/pip/wheels/c6/24/8b/be911b059a61f490f38425eb19bf2fed470a5ead97228e8255
Successfully built semopy


In [ ]:
# BAYESIAN NETWORK ANALYSIS
import pandas as pd
import numpy as np
import warnings
import time

try:
    import bnlearn as bn
except ImportError:
    print("Error: bnlearn not found. Please install it first: pip install bnlearn")
    exit()

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, balanced_accuracy_score, accuracy_score

warnings.filterwarnings('ignore')

# --- 1. Load the Pre-processed Data ---
print("--- 1. Loading Optimized Data from Script 1 ---")
try:
    X_train_final = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/X_train_final.csv")
    X_test_final = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/X_test_final.csv")
    y_train = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/y_train.csv")['label'].values
    y_test = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/y_test.csv")['label'].values

    # Combine into a single training DataFrame for bnlearn
    train_df = X_train_final.copy()
    train_df['label'] = y_train

    # bnlearn requires all column names to be strings
    train_df.columns = train_df.columns.astype(str)
    X_test_final.columns = X_test_final.columns.astype(str)

    print("Data loaded successfully.\n")
except FileNotFoundError:
    print("Error: Could not find data files. Please run Script 1 first.")
    exit()

# --- 2. Main Structure Learning (The Slowest Step, Done Once) ---
print("--- 2. Starting Main Structure Learning (this will take a long time) ---")
print("The script will learn the network graph from the entire training dataset.")
start_time_structure = time.time()

# Learn the Directed Acyclic Graph (DAG) structure using Hill Climbing
main_structure = bn.structure_learning.fit(train_df, methodtype='hc', scoretype='bic')

print(f"Structure learning finished in {(time.time() - start_time_structure) / 60:.2f} minutes.\n")

# --- 3. Main Parameter Learning ---
print("--- 3. Learning Parameters for the Final Model ---")
# Learn the Conditional Probability Distributions (CPDs) for the learned structure
final_model_bn = bn.parameter_learning.fit(main_structure, train_df, methodtype='maximumlikelihood')
print("Final model created.\n")

# --- 4. Efficient Stability Calculation (CV) ---
print("--- 4. Calculating Stability (CV Standard Deviation) ---")
print("This will reuse the learned structure for speed.")
start_time_cv = time.time()

cv_scores_bn = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(skf.split(X_train_final, y_train)):
    print(f"  - Processing Fold {fold+1}/5...")

    # Get the data for this fold
    bn_fold_train = train_df.iloc[train_idx]
    bn_fold_test = train_df.iloc[test_idx]

    # EFFICIENT STEP: Relearn parameters only, using the main structure. This is fast.
    model_fold = bn.parameter_learning.fit(main_structure, bn_fold_train)

    # Predict on the validation part of the fold
    preds_fold = bn.predict(model_fold, bn_fold_test.drop('label', axis=1), variables=['label'])['label']
    cv_scores_bn.append(accuracy_score(bn_fold_test['label'], preds_fold))

cv_std_dev = np.std(cv_scores_bn)
print(f"Stability calculation finished in {(time.time() - start_time_cv) / 60:.2f} minutes.\n")


# --- 5. Final Evaluation on the Test Set ---
print("--- 5. Evaluating Final Model on Unseen Test Data ---")
bn_preds = bn.predict(final_model_bn, X_test_final, variables=['label'])['label']

report_bn = classification_report(y_test, bn_preds, output_dict=True, zero_division=0)
bal_acc_bn = balanced_accuracy_score(y_test, bn_preds)

results = {
    'Accuracy': report_bn['accuracy'],
    'Macro_F1_Score': report_bn['macro avg']['f1-score'],
    'Balanced_Accuracy': bal_acc_bn,
    'CV_Std_Dev': cv_std_dev
}

# --- 6. Display Final Results ---
print("\n--- BAYESIAN NETWORK RESULTS ---")
print("Performance on the unseen test set & stability on the training set:\n")
print(f"{'Bayesian Network':<20} | Acc: {results['Accuracy']:.4f} | Macro F1: {results['Macro_F1_Score']:.4f} | Bal Acc: {results['Balanced_Accuracy']:.4f} | CV Std: {results['CV_Std_Dev']:.4f}")

--- 1. Loading Optimized Data from Script 1 ---
Data loaded successfully.

--- 2. Starting Main Structure Learning (this will take a long time) ---
The script will learn the network graph from the entire training dataset.
[bnlearn] >Computing best DAG using [hc]
[bnlearn] >Set scoring type at [bic]


Benchmarking Models (w/out Bayesian Network)

In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
import os

warnings.filterwarnings('ignore')

# --- 1. Load Train/Test Data ---
print("--- 1. Loading Train/Test Data ---")
try:
    X_train_final = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/X_train_final.csv").values
    y_train = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/y_train.csv")['label'].values
    X_test_final = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/X_test_final.csv").values
    y_test = pd.read_csv("/content/drive/MyDrive/paneer/csv_files/y_test.csv")['label'].values
    print("Train and Test data loaded successfully.\n")
except FileNotFoundError as e:
    print("Error loading CSV files:", e)
    raise

# --- 2. Load Models and Evaluate ---
print("--- 2. Loading Saved Models and Evaluating ---")
model_names = [
    "Logistic Regression", "KNN", "SVM", "Decision Tree",
    "Random Forest", "AdaBoost", "MLP" , "XGBoost"
]

results = {}
model_folder = "/content/drive/MyDrive/paneer/models"

for name in model_names:
    filename = os.path.join(model_folder, f"{name.replace(' ', '_')}_best_model.pkl")
    print(f"Checking {name}: {filename}")
    if os.path.exists(filename):
        print(f"Loading {name}...")
        model = joblib.load(filename)
        cv_scores = cross_val_score(model, X_train_final, y_train, cv=5, n_jobs=-1)
        preds = model.predict(X_test_final)
        report = classification_report(y_test, preds, output_dict=True, zero_division=0)
        bal_acc = balanced_accuracy_score(y_test, preds)
        results[name] = {
            'Accuracy': report['accuracy'],
            'Macro_F1_Score': report['macro avg']['f1-score'],
            'Balanced_Accuracy': bal_acc,
            'CV_Std_Dev': cv_scores.std()
        }
        print(f"{name} loaded and evaluated.\n")
    else:
        results[name] = "Model file not found. Please run the tuner script first."
        print(f"{name} model file not found.\n")

# --- 3. Evaluate Naive Bayes (train fresh) ---
print("Evaluating Naive Bayes...")
nb_model = GaussianNB().fit(X_train_final, y_train)
nb_preds = nb_model.predict(X_test_final)
nb_cv_scores = cross_val_score(GaussianNB(), X_train_final, y_train, cv=5, n_jobs=-1)
nb_report = classification_report(y_test, nb_preds, output_dict=True, zero_division=0)
nb_bal_acc = balanced_accuracy_score(y_test, nb_preds)
results["Naive Bayes"] = {
    'Accuracy': nb_report['accuracy'],
    'Macro_F1_Score': nb_report['macro avg']['f1-score'],
    'Balanced_Accuracy': nb_bal_acc,
    'CV_Std_Dev': nb_cv_scores.std()
}

# --- 4. Display Final Sorted Results ---
print("\n--- FINAL BENCHMARK RESULTS ---")
sorted_results = sorted(results.items(), key=lambda item: item[1]['Macro_F1_Score'] if isinstance(item[1], dict) else -1, reverse=True)
for name, metrics in sorted_results:
    if isinstance(metrics, dict):
        print(f"{name:<20} | Acc: {metrics['Accuracy']:.4f} | Macro F1: {metrics['Macro_F1_Score']:.4f} | "
              f"Bal Acc: {metrics['Balanced_Accuracy']:.4f} | CV Std: {metrics['CV_Std_Dev']:.4f}")
    else:
        print(f"{name:<20} | {metrics}")


--- 1. Loading Train/Test Data ---
Train and Test data loaded successfully.

--- 2. Loading Saved Models and Evaluating ---
Checking Logistic Regression: /content/drive/MyDrive/paneer/models/Logistic_Regression_best_model.pkl
Loading Logistic Regression...
Logistic Regression loaded and evaluated.

Checking KNN: /content/drive/MyDrive/paneer/models/KNN_best_model.pkl
Loading KNN...
KNN loaded and evaluated.

Checking SVM: /content/drive/MyDrive/paneer/models/SVM_best_model.pkl
Loading SVM...
SVM loaded and evaluated.

Checking Decision Tree: /content/drive/MyDrive/paneer/models/Decision_Tree_best_model.pkl
Loading Decision Tree...
Decision Tree loaded and evaluated.

Checking Random Forest: /content/drive/MyDrive/paneer/models/Random_Forest_best_model.pkl
Loading Random Forest...
Random Forest loaded and evaluated.

Checking AdaBoost: /content/drive/MyDrive/paneer/models/AdaBoost_best_model.pkl
Loading AdaBoost...
AdaBoost loaded and evaluated.

Checking MLP: /content/drive/MyDrive/pan